In [1]:
using DifferentialEquations
using Plots
using Plots.PlotMeasures

In [ ]:
# Initial conditions function
function initial_conditions(x, y, Lx, Ly)
    if Ly-5 <= y <= Ly
        if abs(x - (Lx ÷ 2)) < Lx / 50
            n1 = 1.0  # Type 1
            n2 = 0.0
        else
            n1 = 0.0
            n2 = 1.0  # Type 2
        end
    else
        n1 = 0.0
        n2 = 0.0
    end
    return n1, n2
end

# Laplacian function
function laplacian(u, dx, dy)
    lx = (circshift(u, (0, 1)) .- 2u .+ circshift(u, (0, -1))) / dx^2
    ly = (circshift(u, (1, 0)) .- 2u .+ circshift(u, (-1, 0))) / dy^2

    lx[:, 1] .= (u[:, 2] .- u[:, 1]) / dx^2
    lx[:, end] .= (u[:, end-1] .- u[:, end]) / dx^2
    ly[1, :] .= (u[2, :] .- u[1, :]) / dy^2
    ly[end, :] .= (u[end-1, :] .- u[end, :]) / dy^2

    return lx + ly
end

function laplacian_x(u, dx, dy)
    lx = (circshift(u, (0, 1)) .- 2u .+ circshift(u, (0, -1))) / dx^2

    lx[:, 1] .= (u[:, 2] .- u[:, 1]) / dx^2
    lx[:, end] .= (u[:, end-1] .- u[:, end]) / dx^2

    return lx
end

function laplacian_y(u, dx, dy)
    ly = (circshift(u, (1, 0)) .- 2u .+ circshift(u, (-1, 0))) / dy^2

    ly[1, :] .= (u[2, :] .- u[1, :]) / dy^2
    ly[end, :] .= (u[end-1, :] .- u[end, :]) / dy^2

    return ly
end

# Partial derivatives functions
function ∂x(u, dx)
    du = (circshift(u, (0, -1)) .- circshift(u, (0, 1))) / (2dx)
    du[:, 1] .= (u[:, 2] .- u[:, 1]) / dx
    du[:, end] .= (u[:, end] .- u[:, end-1]) / dx
    return du
end

function ∂y(u, dy)
    du = (circshift(u, (-1, 0)) .- circshift(u, (1, 0))) / (2dy)
    du[1, :] .= (u[2, :] .- u[1, :]) / dy
    du[end, :] .= (u[end, :] .- u[end-1, :]) / dy
    return du
end

# Main simulation function
function simulate_reaction_diffusion(; r1=0.15, r2=0.15, K=0.1, D = 0.01, D1x=0.2, D1y=0.2, D2x=0.2, D2y=0.2, a11 = 0.0, a21 = 0.0, a12 = 0.0, a22 = 0.0, 
                                     Lx=100.0, Ly=100.0, Nx=200, Ny=200, t_final = 100.0, num_times_saved=1)

    # Grid
    x = LinRange(0, Lx, Nx)
    y = LinRange(0, Ly, Ny)
    dx = x[2] - x[1]
    dy = y[2] - y[1]

    # Reaction-diffusion system function
    function reaction_diffusion!(du, u, p, t)
        n1 = u[1, :, :]
        n2 = u[2, :, :]
        n = n1 .+ n2
        f = n1 ./ (n .+ 1e-5)

        du[1, :, :] .= r1 .* (1 .+ a11 .* f .+ a12 .* (1 .- f)) .* n1 .* (1 .- n) .+ 
                    #    D1x .* laplacian_x(n1, dx, dy) .+ D1y .* laplacian_y(n1, dx, dy) 
                       D1x .* ∂x(n .* ∂x(n1, dx), dx) + D1y .* ∂y(n .* ∂y(n1, dy), dy) + D * laplacian(n1, dx, dy)
        du[2, :, :] .= r2 .* (1 .+ a21 .* f .+ a22 .* (1 .- f)) .* n2 .* (1 .- n) .+  
                    #    D2x .* laplacian_x(n2, dx, dy) .+ D2y .* laplacian_y(n2, dx, dy) 
                       D2x .* ∂x(n .* ∂x(n2, dx), dx) + D2y .* ∂y(n .* ∂y(n2, dy), dy) + D * laplacian(n2, dx, dy)
    end

    # Initialize conditions
    u0 = zeros(2, Ny, Nx)
    for i in 1:Nx
        for j in 1:Ny
            n1, n2 = initial_conditions(x[i], y[j], Lx, Ly)
            u0[1, j, i] = n1
            u0[2, j, i] = n2
        end
    end

    # PDE problem
    prob = ODEProblem(reaction_diffusion!, u0, (0.0, t_final))

    # Solve
    sol = solve(prob, Tsit5(); saveat=t_final / num_times_saved)

    # Return the final solution
    return sol
end

function simulate_reaction_diffusion_quad(; r1=0.15, r2=0.15, K=0.1, D = 0.01, D1x=0.2, D1y=0.2, D2x=0.2, D2y=0.2, a11 = 0.0, a21 = 0.0, a12 = 0.0, a22 = 0.0, 
                                     Lx=100.0, Ly=100.0, Nx=200, Ny=200, t_final = 100.0, num_times_saved=1)

    # Grid
    x = LinRange(0, Lx, Nx)
    y = LinRange(0, Ly, Ny)
    dx = x[2] - x[1]
    dy = y[2] - y[1]

    # Reaction-diffusion system function
    function reaction_diffusion!(du, u, p, t)
        n1 = u[1, :, :]
        n2 = u[2, :, :]
        n = n1 .+ n2
        f = n1 ./ (n .+ 1e-5)

        du[1, :, :] .= r1 .* (1 .+ a11 .* f .+ a12 .* (1 .- f)) .* n1 .* (1 .- n) .+ 
                    #    D1x .* laplacian_x(n1, dx, dy) .+ D1y .* laplacian_y(n1, dx, dy) 
                       D1x .* ∂x(n .* (1 .- n) .* ∂x(n1, dx), dx) + D1y .* ∂y(n .* (1 .- n) .* ∂y(n1, dy), dy) + D * laplacian(n1, dx, dy)
        du[2, :, :] .= r2 .* (1 .+ a21 .* f .+ a22 .* (1 .- f)) .* n2 .* (1 .- n) .+  
                    #    D2x .* laplacian_x(n2, dx, dy) .+ D2y .* laplacian_y(n2, dx, dy) 
                       D2x .* ∂x(n .* (1 .- n) .* ∂x(n2, dx), dx) + D2y .* ∂y(n .* (1 .- n) .* ∂y(n2, dy), dy) + D * laplacian(n2, dx, dy)
    end

    # Initialize conditions
    u0 = zeros(2, Ny, Nx)
    for i in 1:Nx
        for j in 1:Ny
            n1, n2 = initial_conditions(x[i], y[j], Lx, Ly)
            u0[1, j, i] = n1
            u0[2, j, i] = n2
        end
    end

    # PDE problem
    prob = ODEProblem(reaction_diffusion!, u0, (0.0, t_final))

    # Solve
    sol = solve(prob, Tsit5(); saveat=t_final / num_times_saved)

    # Return the final solution
    return sol
end

# Main simulation function
function simulate_reaction_diffusion_linear(; r1=1.0, r2=1.0, K=0.1, D = 0.0, D1x=0.01, D1y=0.01, D2x=0.01, D2y=0.01, a11 = 0.0, a21 = 0.0, a12 = 0.0, a22 = 0.0, 
                                     Lx=100.0, Ly=100.0, Nx=200, Ny=200, t_final = 100.0, num_times_saved=1)

    # Grid
    x = LinRange(0, Lx, Nx)
    y = LinRange(0, Ly, Ny)
    dx = x[2] - x[1]
    dy = y[2] - y[1]

    # Reaction-diffusion system function
    function reaction_diffusion!(du, u, p, t)
        n1 = u[1, :, :]
        n2 = u[2, :, :]
        n = n1 .+ n2
        f = n1 ./ (n .+ 1e-5)

        du[1, :, :] .= r1 .* (1 .+ a11 .* f .+ a12 .* (1 .- f)) .* n1 .* (1 .- n) .* (n .- K) .+ 
                       D1x .* laplacian_x(n1, dx, dy) .+ D1y .* laplacian_y(n1, dx, dy) 
                    #    D1x .* ∂x(n .* (1 .- n) .* ∂x(n1, dx), dx) + D1y .* ∂y(n .* (1 .- n) .* ∂y(n1, dy), dy) + D * laplacian(n1, dx, dy)
        du[2, :, :] .= r2 .* (1 .+ a21 .* f .+ a22 .* (1 .- f)) .* n2 .* (1 .- n) .* (n .- K) .+  
                       D2x .* laplacian_x(n2, dx, dy) .+ D2y .* laplacian_y(n2, dx, dy) 
                    #    D2x .* ∂x(n .* (1 .- n) .* ∂x(n2, dx), dx) + D2y .* ∂y(n .* (1 .- n) .* ∂y(n2, dy), dy) + D * laplacian(n2, dx, dy)
    end

    # Initialize conditions
    u0 = zeros(2, Ny, Nx)
    for i in 1:Nx
        for j in 1:Ny
            n1, n2 = initial_conditions(x[i], y[j], Lx, Ly)
            u0[1, j, i] = n1
            u0[2, j, i] = n2
        end
    end

    # PDE problem
    prob = ODEProblem(reaction_diffusion!, u0, (0.0, t_final))

    # Solve
    sol = solve(prob, Tsit5(); saveat=t_final / num_times_saved)

    # Return the final solution
    return sol
end

using Colors

function create_colormap(f1, total_density)
    light_sky_blue = RGB(0/255, 174/255, 239/255)
    yellow = RGB(249/255, 237/255, 50/255)
    colormap = [
        RGB(
            yellow.r * (1 - f1[i, j]) + light_sky_blue.r * f1[i, j],
            yellow.g * (1 - f1[i, j]) + light_sky_blue.g * f1[i, j],
            yellow.b * (1 - f1[i, j]) + light_sky_blue.b * f1[i, j]
        ) * total_density[i, j]
        for i in 1:size(f1, 1), j in 1:size(f1, 2)
    ]
    return colormap
end

function create_colormap(f1, total_density)
    light_sky_blue = RGB(0/255, 174/255, 239/255)
    yellow = RGB(249/255, 237/255, 50/255)
    white = RGB(1, 1, 1)  # Color for low-density regions

    colormap = [
        total_density[i, j] < 0.3 ? white :  # Set to white if density is low
        RGB(
            yellow.r * (1 - f1[i, j]) + light_sky_blue.r * f1[i, j],
            yellow.g * (1 - f1[i, j]) + light_sky_blue.g * f1[i, j],
            yellow.b * (1 - f1[i, j]) + light_sky_blue.b * f1[i, j]
        ) 
        for i in 1:size(f1, 1), j in 1:size(f1, 2)
    ]
    return colormap
end

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=8, radius_fraction=0.1)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = hypot(x - center_x, y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor(angle / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 1.0, 0.0  # Type 1
        else
            n1, n2 = 0.0, 1.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end


solution = simulate_reaction_diffusion(r1=0.25, r2=0.25, a11=0.0, a21=0.0, a12=0.0, a22=0.0,
                                        D=0.01, D1x=0.05, D1y=0.2, D2x=0.05, D2y=0.2,
                                        Lx=150.0, Ly=150.0, Nx=400, Ny=400, t_final=300.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)
filename = "other_rd_figs/neutral_nonrescale_aniso"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=8, radius_fraction=0.1)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = hypot(x - center_x, y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor(angle / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 1.0, 0.0  # Type 1
        else
            n1, n2 = 0.0, 1.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end


solution = simulate_reaction_diffusion(r1=0.25, r2=0.25, a11=0.0, a21=0.0, a12=0.0, a22=0.0,
                                        D=0.0, D1x=0.05, D1y=0.2, D2x=0.05, D2y=0.2,
                                        Lx=150.0, Ly=150.0, Nx=400, Ny=400, t_final=300.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)
filename = "other_rd_figs/neutral_rescale_aniso"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=8, radius_fraction=0.1)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = hypot(x - center_x, y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor(angle / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 1.0, 0.0  # Type 1
        else
            n1, n2 = 0.0, 1.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end


solution = simulate_reaction_diffusion(r1=0.25, r2=0.25, a11=0.0, a21=0.0, a12=0.0, a22=0.0,
                                        D=0.02, D1x=0.02, D1y=0.2, D2x=0.02, D2y=0.2,
                                        Lx=150.0, Ly=150.0, Nx=400, Ny=400, t_final=100.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)
filename = "other_rd_figs/neutral_verynonrescale_aniso"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
# Initial conditions function
function initial_conditions(x, y, Lx, Ly)
    if Ly-5 <= y <= Ly
        if abs(x - (Lx ÷ 2)) < Lx / 50
            n1 = 1.0  # Type 1
            n2 = 0.0
        else
            n1 = 0.0
            n2 = 1.0  # Type 2
        end
    else
        n1 = 0.0
        n2 = 0.0
    end
    return n1, n2
end

solution = simulate_reaction_diffusion(r1=0.35, r2=0.35, a11=0.6, a21=0.6, a12=0.0, a22=0.0,
                                        D=0.0, D1x=0.03, D1y=0.03, D2x=0.05, D2y=0.03,
                                        Lx=150.0, Ly=150.0, Nx=300, Ny=300, t_final=700.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
p1 = heatmap(colormap, axis=nothing, frame=:none, size=(800, 300), margins=0mm, title="D(n) = n")
filename = "other_rd_figs/linear_diffusivity_escaping"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
# Initial conditions function
function initial_conditions(x, y, Lx, Ly)
    if Ly-5 <= y <= Ly
        if abs(x - (Lx ÷ 2)) < Lx / 50
            n1 = 1.0  # Type 1
            n2 = 0.0
        else
            n1 = 0.0
            n2 = 1.0  # Type 2
        end
    else
        n1 = 0.0
        n2 = 0.0
    end
    return n1, n2
end

solution = simulate_reaction_diffusion_quad(r1=0.35, r2=0.35, a11=0.8, a21=0.8, a12=0.0, a22=0.0,
                                        D=0.001, D1x=0.03, D1y=0.03, D2x=0.05, D2y=0.03,
                                        Lx=150.0, Ly=150.0, Nx=300, Ny=300, t_final=700.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
p2 = heatmap(colormap, axis=nothing, frame=:none, size=(800, 300), margins=0mm, title="D(n) = n(1-n)")
filename = "other_rd_figs/quad_diffusivity_escaping"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
plot(p1,p2)
filename = "other_rd_figs/combined_escaping"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")